# Importing Data

## Importing Libraries

In [ ]:
!pip install flask flask-ngrok
!pip install pyngrok

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from datetime import datetime
from flask import Flask, request, jsonify, send_file
from collections import defaultdict
import requests
import io
from flask_ngrok import run_with_ngrok
import pyngrok.ngrok as ngrok

## Setting URLs and creating DataFrames

In [ ]:
url_1 = "https://raw.githubusercontent.com/thais-menezes/monitoring/main/transactions_1.csv"
url_2 = "https://raw.githubusercontent.com/thais-menezes/monitoring/main/transactions_2.csv"

In [ ]:
df_1 = pd.read_csv(url_1)
df_2 = pd.read_csv(url_2)

In [ ]:
df_1.head()

,time,status,f0_
0,00h 00,approved,9
1,00h 00,denied,6
2,00h 00,refunded,1
3,00h 01,denied,8
4,00h 01,approved,13


In [ ]:
df_2.head()

,time,status,count
0,00h 00,reversed,7
1,00h 00,approved,9
2,00h 00,processing,12
3,00h 00,denied,3
4,00h 00,backend_reversed,2


# Preparing Trigger Alert and ML

## Preparing Data

In [ ]:
def aggregate_date(df):
  # Aggregate data by 'time' and 'status'
  df = df.groupby(['time', 'status']).size().unstack(fill_value=0)
  # Total by 'time'
  df['total'] = df.sum(axis=1)
  # Reseting index for visualization
  df = df.reset_index()

In [ ]:
aggregate_date(df_1)
aggregate_date(df_2)

In [ ]:
def create_flags(df):
  # Creating flags (alerts) with conditions
  df['alert_failed'] = df['failed'] > 0.05 * df['total']
  df['alert_denied'] = df['denied'] > 10
  df['alert_reversed'] = df['reversed'] > 5
  # Check where any alert is triggered
  df['rule_alert'] = df[['alert_failed', 'alert_denied', 'alert_reversed']].any(axis=1)

In [ ]:
create_flags(df_1)
create_flags(df_2)

## Training Model (ML Isolation Forest)

In [ ]:
def create_isolation_forest(df):
  features = df[['failed', 'denied', 'reversed', 'approved']]
  # Training Isolation Forest
  model = IsolationForest(contamination=0.05, random_state=42)
  df['anomaly_score'] = model.fit_predict(features)
  # Isolation Forest returns -1 for anomalies
  df['model_alert'] = df['anomaly_score'] == -1

In [ ]:
create_isolation_forest(df_1)
create_isolation_forest(df_2)

## Defining Alert Trigger (Rule-Based + Score-Based)

In [ ]:
def final_alert(df):
  # Define final_alert (rule-based + score-based)
  df['final_alert'] = df['rule_alert'] | df['model_alert']

In [ ]:
final_alert(df_1)
final_alert(df_2)

## Plotting Graphic

In [ ]:
# Plotting Failed Transactions Graphic
def plot_failed_transactions(df):
    plt.figure(figsize=(15, 6))
    sns.lineplot(data=df, x='time', y='failed', label='Failed')
    sns.scatterplot(data=df[df['final_alert']], x='time', y='failed', color='red', label='Alerts')
    plt.title("Failed Transactions")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_failed_transactions(df_1)

In [ ]:
plot_failed_transactions(df_2)

# Notification via Teams

## Creating APP and URL

In [ ]:
# Creating Flask app
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
# Link to Teams URL. Paste it on the final.
teams_webhook_url = "https://outlook.office.com/webhook/YOUR-WEBHOOK-URL-HERE"

## Creating Real-Time Image

In [ ]:
# Create public URL for image
@app.before_first_request
def init_ngrok_url():
    global public_url
    public_url = ngrok.get_tunnels()[0].public_url

In [ ]:
# Function to plot image
def save_failed_transactions_plot(df, filename="alert_plot.png"):
    plt.figure(figsize=(15, 6))
    sns.lineplot(data=df, x='time', y='failed', label='Failed')
    sns.scatterplot(data=df[df['final_alert']], x='time', y='failed', color='red', label='Alerts')
    plt.title("Failed Transactions")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

In [ ]:
# Route to serve this file publicly
@app.route("/plot.png")
def serve_plot():
    return send_file("alert_plot.png", mimetype="image/png")

## Creating Message

In [ ]:
# Function to create Teams Message
def send_teams_notification(time, failed, denied, reversed_val, rule_alert, model_alert, image_url):
  headers = {
      'Content-Type': 'application/json'
  }

  # Message with the values
  message = f"""
              **🚨 Transaction Alert Detected!**

              > **🕒 Time:** `{time}`
              > **❌ Failed:** `{failed}`
              > **🚫 Denied:** `{denied}`
              > **🔁 Reversed:** `{reversed_val}`

              **Alert Types:**
              - 🧠 **ML Alert:** `{model_alert}`
              - 📜 **Rule Alert:** `{rule_alert}`

              📊 **Plot:**
              ![Failed Transactions Plot]({image_url})

              ---

              📩 _This message was automatically generated by our monitoring system._
            """

  payload = {
      "text": message
  }

  response = requests.post(teams_webhook_url, headers=headers, json=payload)
  return response.status_code

## Creating Endpoint

In [ ]:
# Route to send Teams Message
@app.route("/check_alert", methods=["POST"])
def check_alert():
  # Procesing and Transforming data
  data = request.get_json()
  df = pd.DataFrame(data)

  aggregate_date(df)
  create_flags(df)
  create_isolation_forest(df)
  final_alert(df)
  save_failed_transactions_plot(df)

  # Generate public image URL
  image_url = f"{public_url}/plot.png"

  # Send Teams Message if there is(are) alert(s)
  alerts = df[df["final_alert"]]

  if not alerts.empty:
    for _, row in alerts.iterrows():
      send_teams_notification(
        time=row['time'],
        failed=row['failed'],
        denied=row['denied'],
        reversed_val=row['reversed'],
        rule_alert=row['rule_alert'],
        model_alert=row['model_alert'],
        image_url=image_url
      )

    return jsonify({"status": "Alerts sent to Teams!"}), 200
  else:
    return jsonify({"status": "No alerts found."}), 200

## Running

In [ ]:
# Run the app
app.run()